# Task 2

In [1]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer,IBMQ
from scipy.optimize import minimize
import numpy as np

# Noisy Model

In [2]:
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_essex')
noise_model = NoiseModel.from_backend(backend)
coupling_map = backend.configuration().coupling_map
basis_gates = noise_model.basis_gates

# Defining Ansatz and Cost Functions

# 1. Ansatz

In [7]:
shots= 1024
def ansatz_counts(params):
    qc = QuantumCircuit(2,2)

    #q = QuantumRegister(2, 'q')

    #qc.add_register(q)
    #qc.h(0)
    #qc.h(1)
    qc.rx(params[4],0)
    qc.rx(params[5],1)
    qc.ry(params[0], 0)
    qc.ry(params[1], 1)
    qc.cx(0, 1)
    qc.ry(params[2], 0)
    qc.ry(params[3], 1)
    
    qc.measure(range(2), range(2))
    #shots = 1000
    
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend,
                 coupling_map=coupling_map,
                 basis_gates=basis_gates,
                 noise_model=noise_model, shots= shots)
    job_result = job.result()
    counts = job_result.get_counts(qc)
    return counts

# 2. Cost_Function

In [8]:
def cost_function(params):
    counts = ansatz_counts(params)
    # CALCULATE COST HERE
    cost = 0
    cost += np.abs((counts["01"] if "01" in counts else 0) / shots - 0.5)
    #print(cost)
    cost += np.abs((counts["10"] if "10" in counts else 0) / shots - 0.5)
    #print(cost)
    cost += np.abs((counts["11"] if "11" in counts else 0) / shots )
    #print(cost)
    cost += np.abs((counts["00"] if "00" in counts else 0) / shots )
    #print(cost)


    return cost#,counts

In [9]:
params = np.array([0,0,0,0,0,0]) # Initial_Params

In [10]:
print(cost_function(params))     # Try_Run

1.896484375


# Optimization for 1024 Counts per iteration

In [11]:
from qiskit.aqua.components.optimizers import COBYLA,SLSQP,SPSA

optimizer = COBYLA(100,0.001)

In [12]:
ret = optimizer.optimize(num_vars=6, objective_function=cost_function, initial_point=params)

In [14]:
print(ansatz_counts(ret[0]))
print(cost_function(ret[0]))

{'11': 25, '00': 49, '10': 459, '01': 491}
0.197265625


# Sampling For Counts = [1,10,100,1000]

In [15]:
shots_array = [1,10,100,1000]


In [16]:
result = []
cost = []
counts= []
#optimizer = SPSA(500)
for i in range (len(shots_array)):
    shots = shots_array[i]
    ret_loop= optimizer.optimize(num_vars=6, objective_function=cost_function, initial_point=params)
    result.append([shots,cost_function(ret_loop[0]),ansatz_counts(ret_loop[0])])
    cost.append(cost_function(ret_loop[0]))
    counts.append(ansatz_counts(ret_loop[0]))
    
    

In [17]:
print(result)

[[1, 2.0, {'00': 1}], [10, 0.6000000000000001, {'11': 2, '00': 1, '10': 6, '01': 1}], [100, 0.22000000000000003, {'11': 2, '00': 14, '10': 34, '01': 50}], [1000, 0.276, {'11': 34, '00': 86, '10': 405, '01': 475}]]


In [18]:
print(cost)

[2.0, 0.19999999999999998, 0.14, 0.19599999999999998]


In [19]:
print(counts)

[{'00': 1}, {'11': 2, '00': 1, '10': 4, '01': 3}, {'11': 4, '00': 10, '10': 38, '01': 48}, {'11': 44, '00': 73, '10': 382, '01': 501}]


# Result

We can see as the cost suggest that we can't get to a result with a single measurment of our circuit. And to get the best we need at least 100 measurment per iteration.